In [ ]:
%tensorflow_version 2.x
import tensorflow
tensorflow.__version__



In [ ]:
import random
import os 
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
  
from PIL import Image
import numpy as np

import tensorflow as tf
import os
import pandas as pd

random.seed(0)

# Ignore the warnings
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
ls "/content/drive/MyDrive/Dataset/Car Images/"


In [ ]:
dataset_loc="/content/drive/MyDrive/"
train_annotation = pd.read_csv(dataset_loc+"/Dataset/Annotations/Train Annotations.csv")
train_annotation.info()

In [ ]:
entry = train_annotation.loc[train_annotation['Image Name'] == '01286.jpg'].values[0]
#dim_analysis.loc[dim_analysis['dim3'] !=3 ]

In [ ]:
entry[1]

In [ ]:
train_annotation.rename(columns = {'Bounding Box coordinates':'x0', 'Unnamed: 2':'yo', 'Unnamed: 3':'x1', 'Unnamed: 4':'y1'}, inplace = True)
   

In [ ]:
train_annotation.loc[train_annotation['Image Name'] == '00134.jpg']

In [ ]:
#train_images.rename(columns = {'class':'label'}, inplace = True)

In [ ]:
train_annotation

In [ ]:


train_images = pd.DataFrame(columns=['image','image_matrix','filename','car', 'x0', 'y0', 'x1', 'y1', 'label'])
dim_analysis = pd.DataFrame(columns=['filename', 'dim1', 'dim2', 'dim3'])


base=dataset_loc+"/Dataset/Car Images/Train Images"
newsize = (128, 128)
reshape_size = 128*128*3
size_dim=128


index = 0
for dirs in os.listdir(base):
    full_dir=base+"/"+dirs
    
    if os.path.isfile(full_dir):
      continue
    for file in os.listdir(os.path.join(full_dir)):
      
      if os.path.isfile(os.path.join(full_dir, file)):
        filename=full_dir+"/"+file
        im = Image.open(filename, mode='r')
        dim_path=dirs+"/"+filename
        image_width = im.size[1]
        image_height = im.size[0]
        dim_analysis = dim_analysis.append({'filename': filename, 'dim1':image_height, 'dim2':image_width}, ignore_index = True)
  
            #print(im.size[0])
        im = im.resize(newsize)
        pix_val = np.array(list(im.getdata()))
        #print("\t \t", len(pix_val.shape))
        if ((len(pix_val.shape) >= 2) and (pix_val.shape[0]*pix_val.shape[1] == reshape_size)):
          bound_box = train_annotation.loc[train_annotation['Image Name'] == file].values[0]
          x0=bound_box[1]
          x0 = int((x0 / image_width) * size_dim) 

          y0=bound_box[2]
          y0 = int((y0 / image_height ) * size_dim)

          x1=bound_box[3]
          x1 = int((x1 / image_width)  * size_dim)

          y1=bound_box[4]
          y1 = int((y1 / image_height) * size_dim)
          train_images = train_images.append({'image': pix_val.reshape(reshape_size), 'image_matrix': pix_val, 'filename':full_dir+"/"+file, 'car':dirs, 'x0': x0, 'y0':y0, 'x1':x1, 'y1':y1, 'label':bound_box[5]}, ignore_index = True)
          


In [ ]:
train_images

In [ ]:
dim_analysis.loc[0, 'filename']

In [ ]:
print("Various different dimensions of the image: \n ", np.unique(dim_analysis['dim1']))

In [ ]:
import seaborn as sns
print("Dimension Analysis:\n")
val_count = dim_analysis['dim1'].value_counts()
print(val_count)


In [ ]:

sns.histplot(dim_analysis['dim1'])

In [ ]:
val_dict = dict(zip(train_images.car, train_images.label))

In [ ]:
val_dict

In [ ]:

print("car counts:\n")
val_count = train_images['car'].value_counts()
print(val_count)

In [ ]:
sns.histplot(train_images['car'])

In [ ]:
plt.imshow(im)
plt.show()

In [ ]:
# few statistical analysis
train_images.loc[train_images['car'] == 'GMC Savana Van 2012']


In [ ]:
len_train = train_images.shape[0]

In [ ]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

#Some random images
for i in range( 2, 1000, 100):
    print("=============\n", train_images.loc[i,'car'])
    plt.imshow(train_images.loc[i, 'image'].reshape(size_dim, size_dim, 3))
    plt.show()

In [ ]:
dim_analysis

In [ ]:
def normalising_box(ind):
  test_img = train_images.loc[ind, 'filename']
  test_dim = dim_analysis.loc[dim_analysis['filename'] == test_img].values[0]
  image_width = test_dim[2]
  image_height = test_dim[1]
 
  IMAGE_SIZE=128
  x0=y_train[ind][0]
  y0=y_train[ind][1]

  x1=y_train[ind][2]
  y1=y_train[ind][3]

  print(x0, y0, x1, y1)
  
  x0 = int((x0 / image_width) * IMAGE_SIZE) # Scale the BBox
  y0 = int((y0 / image_height ) * IMAGE_SIZE)

  x1 = int((x1 / image_width)  * IMAGE_SIZE)
  y1 = int((y1 / image_height) * IMAGE_SIZE)
  print(x0, y0, x1, y1)
  #return x0, y0, x1, y1


In [ ]:
import matplotlib.pyplot as plt
#Extract the (flattened) data and class



from tensorflow.keras.applications.mobilenet import preprocess_input

#x_train = train_images['image_matrix']

x_train = np.zeros((len_train, size_dim, size_dim, 3))
y_train = np.zeros((len_train, 5))
#y_train = train_images[5:7]
for i in range(len(train_images)) :
  x_train[i] = preprocess_input(np.array(train_images.loc[i,'image_matrix'].reshape(size_dim, size_dim, 3), dtype=np.float32))
  y_train[i] = train_images.loc[i, 'x0':'label']

In [ ]:
import cv2
import matplotlib.pyplot as plt
import matplotlib.patches as patches

def patch_fn (ind):
  
  
  # Create figure and axes
  fig,ax = plt.subplots(1)
  x0=y_train[ind][0]
  y0=y_train[ind][1]

  x1=y_train[ind][2]
  y1=y_train[ind][3]
  print("Normalised Bounding box: ", x0, y0, x1, y1)
  # Display the image
  ax.imshow(train_images.loc[ind,'image_matrix'].reshape(size_dim,size_dim,3))

  # Create a Rectangle patch
  rect = patches.Rectangle((x0, y0), x1 , y1, linewidth=2, edgecolor='r', facecolor='none')

  # Add the patch to the Axes
  ax.add_patch(rect)

  plt.show()



In [ ]:
#Some random bounding box examples
patch_fn(50)
#normalising_box(50)

In [ ]:
patch_fn(200)

In [ ]:
patch_fn(4000)

In [ ]:
patch_fn(6123)

In [ ]:
x_train.shape, y_train.shape

In [ ]:
x_train[25], y_train[25]

Considering VGG16 to do object detection region and if needed SVM on the flatten layer to detect the class(196 class)

In [ ]:
#TEst images




test_images = pd.DataFrame(columns=['image','image_matrix','filename','car', 'x0', 'y0', 'x1', 'y1', 'label'])
dim_analysis_test = pd.DataFrame(columns=['filename', 'dim1', 'dim2', 'dim3'])


base=dataset_loc+"/Dataset/Car Images/Test Images"
newsize = (128, 128)
reshape_size = 128*128*3
size_dim=128


index = 0
for dirs in os.listdir(base):
    full_dir=base+"/"+dirs
    
    if os.path.isfile(full_dir):
      continue
    for file in os.listdir(os.path.join(full_dir)):
      
      if os.path.isfile(os.path.join(full_dir, file)):
        filename=full_dir+"/"+file
        im = Image.open(filename, mode='r')
        dim_path=dirs+"/"+filename
        image_width = im.size[1]
        image_height = im.size[0]
        dim_analysis_test = dim_analysis_test.append({'filename': filename, 'dim1':image_height, 'dim2':image_width}, ignore_index = True)

        im = im.resize(newsize)
        pix_val = np.array(list(im.getdata()))
        if ((len(pix_val.shape) >= 2) and (pix_val.shape[0]*pix_val.shape[1] == reshape_size)):
          bound_box = train_annotation.loc[train_annotation['Image Name'] == file].values[0]
          x0=bound_box[1]
          x0 = int((x0 / image_width) * size_dim) 

          y0=bound_box[2]
          y0 = int((y0 / image_height ) * size_dim)

          x1=bound_box[3]
          x1 = int((x1 / image_width)  * size_dim)

          y1=bound_box[4]
          y1 = int((y1 / image_height) * size_dim)
          test_images = test_images.append({'image': pix_val.reshape(reshape_size), 'image_matrix': pix_val, 'filename':full_dir+"/"+file, 'car':dirs, 'x0': x0, 'y0':y0, 'x1':x1, 'y1':y1, 'label':bound_box[5]}, ignore_index = True)
          


In [ ]:
test_images

In [ ]:
import seaborn as sns
print("Dimension Analysis:\n")
val_count = dim_analysis_test['dim1'].value_counts()
print(val_count)

sns.histplot(dim_analysis_test['dim1'])

In [ ]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

#Some random images
for i in range( 2, 1000, 100):
    print("=============\n", test_images.loc[i,'car'])
    plt.imshow(test_images.loc[i, 'image'].reshape(size_dim, size_dim, 3))
    plt.show()

In [ ]:
len_test = test_images.shape[0]

In [ ]:
import matplotlib.pyplot as plt
#Extract the (flattened) data and class

from tensorflow.keras.applications.mobilenet import preprocess_input


x_test = np.zeros((len_test, size_dim, size_dim, 3))
y_test = np.zeros((len_test, 5))
#y_train = train_images[5:7]
for i in range(len(test_images)) :
  x_test[i] = preprocess_input(np.array(test_images.loc[i,'image_matrix'].reshape(size_dim, size_dim, 3), dtype=np.float32))
  y_test[i] = test_images.loc[i, 'x0':'label']

In [ ]:
x_train.shape, y_train.shape

In [ ]:
x_test.shape, y_test.shape